# Evolving Ensembles in Multi-objective GeneticProgramming for Classification with Unbalanced Data

### Dataset YEAST

Minority class = 'MIT'

Majority class = other tags

-----------------------------------------

Imports

In [1]:
# Python 3.8.3
import deap # 1.3.1
from deap import gp, creator, base, tools

import numpy as np # 1.18.5

import operator

Defining Constants

In [2]:
# Number of individuals in a population
PNUM = 10

Helper functions for primitive operators

In [3]:
# Protected division
def div(x, y):
    if(y == 0):
        return 1
    else:
        return x/y

# If then operator
def if_then(a, b, c):
    if a < 0:
        return b
    else:
        return c

Helper functions for evaluation

In [4]:
# Evaluation function
def evaluate(individual):
    return (1.1, 1.2) # dummy evaluation results

Import dataset from external file

In [5]:
# Define a filename
filename = "./Datasets/yeast.data"

# Opening and reading the file
with open(filename) as f:
    content = f.read().splitlines()
    
data = [] # data for saving the features from file
names = [] # names of the protein classes
    
# Testing by printing the file
for line in content:
    tmp = line.split()
    tmp.pop(0) # pops the first element - we dont need it
    
    names.append(tmp.pop()) # pops the last element - name of protein
    
    data.append(np.asarray(tmp, float)) # extracting features to data as float values

# testing data
for d in data:
    print(d)

[0.58 0.61 0.47 0.13 0.5  0.   0.48 0.22]
[0.43 0.67 0.48 0.27 0.5  0.   0.53 0.22]
[0.64 0.62 0.49 0.15 0.5  0.   0.53 0.22]
[0.58 0.44 0.57 0.13 0.5  0.   0.54 0.22]
[0.42 0.44 0.48 0.54 0.5  0.   0.48 0.22]
[0.51 0.4  0.56 0.17 0.5  0.5  0.49 0.22]
[0.5  0.54 0.48 0.65 0.5  0.   0.53 0.22]
[0.48 0.45 0.59 0.2  0.5  0.   0.58 0.34]
[0.55 0.5  0.66 0.36 0.5  0.   0.49 0.22]
[0.4  0.39 0.6  0.15 0.5  0.   0.58 0.3 ]
[0.43 0.39 0.54 0.21 0.5  0.   0.53 0.27]
[0.42 0.37 0.59 0.2  0.5  0.   0.52 0.29]
[0.4  0.42 0.57 0.35 0.5  0.   0.53 0.25]
[0.6  0.4  0.52 0.46 0.5  0.   0.53 0.22]
[0.66 0.55 0.45 0.19 0.5  0.   0.46 0.22]
[0.46 0.44 0.52 0.11 0.5  0.   0.5  0.22]
[0.47 0.39 0.5  0.11 0.5  0.   0.49 0.4 ]
[0.58 0.47 0.54 0.11 0.5  0.   0.51 0.26]
[0.5  0.34 0.55 0.21 0.5  0.   0.49 0.22]
[0.61 0.6  0.55 0.21 0.5  0.   0.5  0.25]
[0.45 0.4  0.5  0.16 0.5  0.   0.5  0.22]
[0.43 0.44 0.48 0.22 0.5  0.   0.51 0.22]
[0.73 0.63 0.42 0.3  0.5  0.   0.49 0.22]
[0.43 0.53 0.52 0.13 0.5  0.   0.5

[0.25 0.48 0.45 0.32 0.5  0.   0.4  0.22]
[0.38 0.34 0.55 0.44 0.5  0.   0.48 0.26]
[0.61 0.54 0.54 0.38 0.5  0.   0.54 0.29]
[0.48 0.17 0.37 0.33 0.5  0.   0.44 0.36]
[0.47 0.47 0.5  0.19 0.5  0.   0.51 0.26]
[0.55 0.38 0.52 0.16 0.5  0.   0.49 0.64]
[0.41 0.39 0.61 0.17 0.5  0.   0.5  0.61]
[0.41 0.56 0.53 0.16 0.5  0.   0.55 0.26]
[0.78 0.73 0.27 0.28 0.5  0.   0.56 0.22]
[0.31 0.51 0.38 0.18 0.5  0.   0.46 0.31]
[0.63 0.64 0.49 0.2  0.5  0.   0.52 0.25]
[0.31 0.4  0.31 0.17 0.5  0.   0.52 0.22]
[0.44 0.65 0.49 0.16 0.5  0.   0.48 0.31]
[0.48 0.44 0.46 0.37 0.5  0.   0.46 0.36]
[0.5  0.49 0.46 0.22 0.5  0.   0.53 0.25]
[0.62 0.42 0.47 0.16 0.5  0.   0.52 0.22]
[0.5  0.45 0.52 0.18 0.5  0.   0.52 0.22]
[0.59 0.51 0.47 0.21 0.5  0.   0.47 0.22]
[0.39 0.47 0.49 0.2  0.5  0.   0.53 0.26]
[0.49 0.48 0.52 0.24 0.5  0.   0.5  0.22]
[0.59 0.48 0.49 0.42 0.5  0.   0.52 0.22]
[0.57 0.35 0.5  0.43 0.5  0.   0.53 0.22]
[0.61 0.52 0.5  0.44 0.5  0.   0.53 0.22]
[0.49 0.51 0.52 0.13 0.5  0.   0.5

Creating a primiive set for the classificator expression

In [6]:
# Creating a primitive set
pset = gp.PrimitiveSet("main", arity=8)

# Adding primitive operators
pset.addPrimitive(operator.add, 2)
pset.addPrimitive(operator.sub, 2)
pset.addPrimitive(operator.mul, 2)
pset.addPrimitive(div, 2) 
pset.addPrimitive(if_then, 3)

# Generate an expression where each leaf has the same depth between min and max.
# expr = gp.genFull(pset, min_=1, max_=8) # Create expression of max depth 8 (from paper)
# tree = gp.PrimitiveTree(expr) # Create tree classificator from expression

Creating a fitness function and an individual

In [7]:
# Create a multi objective fitness function with weights between objectives
creator.create("FitnessTmp", base.Fitness, weights=(0.5, 0.5))

# Create an individual of type primitive tree and our fitness function, pset is a previously defined
creator.create("IndividualTree", gp.PrimitiveTree, fitness=creator.FitnessTmp, pset=pset)


Registering individual and population

In [8]:
# Creating toolbox
toolbox = base.Toolbox()

# Register the type of expression the threes are gonna be with min and max depth (8 from paper)
toolbox.register("expr", gp.genFull, pset=pset, min_=1, max_=8) # genFull - Generate an expression where each leaf has the same depth between min and max
toolbox.register("individual", tools.initIterate, creator.IndividualTree, toolbox.expr) # Register individual from expression


#population of type bag
toolbox.register("population", tools.initRepeat, gp.PrimitiveTree, toolbox.individual)
#generating population of n individuals
population = toolbox.population(n=PNUM)

Feeding data to population

In [9]:
for p in population:
    print(p) # Testing
    
    # Compiling the individual 
    classificator = gp.compile(p, pset)
    for d in data:
        res = classificator(*d) # Passing data features as arguments to classificator
        
        # Testing
        print(res)
#         print('minority') if (res >= 0) else print('majority')
    
    print()

if_then(mul(mul(div(sub(div(ARG5, ARG1), if_then(ARG4, ARG2, ARG5)), div(sub(ARG3, ARG7), add(ARG0, ARG6))), sub(div(div(ARG6, ARG3), mul(ARG2, ARG6)), add(add(ARG3, ARG0), mul(ARG3, ARG4)))), div(add(sub(sub(ARG5, ARG6), if_then(ARG3, ARG7, ARG7)), if_then(div(ARG0, ARG2), if_then(ARG6, ARG3, ARG5), add(ARG3, ARG4))), div(add(add(ARG3, ARG5), add(ARG5, ARG3)), add(add(ARG0, ARG5), div(ARG1, ARG0))))), if_then(if_then(mul(sub(add(ARG2, ARG3), mul(ARG6, ARG3)), mul(add(ARG0, ARG2), if_then(ARG6, ARG4, ARG4))), add(div(add(ARG7, ARG4), sub(ARG1, ARG1)), if_then(if_then(ARG5, ARG3, ARG4), add(ARG1, ARG6), sub(ARG4, ARG7))), div(add(if_then(ARG6, ARG5, ARG1), add(ARG0, ARG6)), add(if_then(ARG0, ARG1, ARG5), mul(ARG6, ARG4)))), sub(add(mul(add(ARG7, ARG5), if_then(ARG6, ARG6, ARG4)), add(sub(ARG1, ARG7), if_then(ARG4, ARG2, ARG7))), sub(if_then(if_then(ARG1, ARG4, ARG1), sub(ARG5, ARG6), mul(ARG6, ARG7)), add(if_then(ARG0, ARG2, ARG0), div(ARG0, ARG7)))), add(if_then(if_then(div(ARG6, ARG0)

0.67
0.28
0.39
0.8
0.5
0.42
0.36
0.46
0.4
0.33
0.49
0.38
0.69
0.61
0.78
0.64
0.36
0.36
0.47
0.48
0.47
0.44
0.42
0.3
0.65
0.27
0.45
0.44
0.3
0.4
0.45
0.45
0.59
0.47
0.35
0.47
0.64
0.55
0.67
0.59
0.54
0.58
0.4
0.46
0.53
0.41
0.5
0.4
0.6
0.62
0.35
0.34
0.39
0.48
0.29
0.47
0.39
0.58
0.64
0.37
0.43
0.28
0.59
0.39
0.26
0.51
0.58
0.44
0.41
0.45
0.36
0.42
0.42
0.67
0.49
0.71
0.4
0.4
0.54
0.42
0.57
0.35
0.42
0.26
0.39
0.26
0.31
0.48
0.66
0.21
0.38
0.44
0.31
0.49
0.38
0.39
0.24
0.49
0.54
0.43
0.48
0.31
0.32
0.32
0.59
0.37
0.75
0.61
0.44
0.52
0.63
0.52
0.58
0.72
0.72
0.46
0.57
0.42
0.49
0.42
0.59
0.52
0.5
0.49
0.5
0.5
0.37
0.55
0.67
0.53
0.35
0.52
0.77
0.47
0.73
0.78
0.7
0.47
0.56
0.57
0.56
0.58
0.53
0.34
0.69
0.37
0.41
0.31
0.36
0.82
0.82
0.41
0.38
0.38
0.53
0.46
0.4
0.53
0.42
0.45
0.2
0.51
0.44
0.27
0.55
0.34
0.44
0.59
0.39
0.56
0.36
0.27
0.35
0.42
0.23
0.55
0.36
0.38
0.8
0.6
0.41
0.47
0.43
0.5
0.53
0.54
0.46
0.46
0.51
0.56
0.61
0.37
0.83
0.39
0.52
0.35
0.36
0.78
0.55
0.45
0.45
0.29
0.52
0.71
0

0.0
0.0
0.0
0.0
-0.0
-0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
-0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
-0.0
-0.0
-0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
-0.0
0.0
-0.0
-0.0
0.0
0.0
0.0
0.0
0.0
0.0
-0.0
0.0
-0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
-0.0
-0.0
-0.0
0.0
0.0
0.0
0.0
0.0
-0.0
0.0
0.0
0.0
0.0
0.0
-0.0
0.0
0.0
0.0
0.0
-0.0
0.0
-0.0
0.0
0.0
0.0
0.0
-0.0
0.0
0.0
-0.0
0.0
0.0
0.0
-0.0
0.0
0.0
0.0
0.0
0.0
-0.0
0.0
0.0
-0.0
-0.0
0.0
0.0
0.0
0.0
0.0
-0.0
0.0
-0.0
0.0
0.0
0.0
0.0
0.0
-0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
-0.0
0.0
-0.0
0.0
-0.0
0.0
0.0
-0.0029881381950000027
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
-0.0
0.0
0.0
0.0
0.0
0.0
-0.0
0.0
0.0
-0.0
0.0
-0.0
0.0
0.0
0.0
0.0
-0.0
0.0
0.0
0.004538473200000006
0.0
0.0
0.0
-0.0
0.0
-0.0
0.0
0.0
-0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
-0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
-0.0
0.0
-0.0
0.0
-0.0
0.0
0.0
-0.0
0.0
-0.0
-0.0
0.0
0.0
-0.0
0.0
0.0
-0.0
0.0
-0.0
-0.0
0.0
-0.0
-0.0
0.0
0.0
-0.0
0.0


0.29311310729505374
0.09072108860028001
0.0029819862632805847
0.011278057522787911
0.2395802828485745
0.11183890381458358
-0.06699302368731398
0.07702664591924836
0.12696034383463187
0.013004710802228594
0.028409474593710935
0.01692309997694971
0.027441649706791398
0.09312566010439413
0.1272818240746873
0.06811949784882344
-0.04814541508886858
0.04460022649144986
0.09430882278618496
-0.03213482154396109
-0.03435121983887208
0.2508452115382231
0.20882757437708968
-0.2403167448138096
-0.0956290096678091
0.12038610580251485
0.03557561350532992
0.015588700280403118
0.03690348652876068
-1.044074124761136
0.04679533607671762
-0.27880337031722685
0.12527449264015292
0.04840910043304005
0.012275499957439095
0.09130280553255761
0.041631242762125716
0.25795489999893434
-1.0562905438753327
-0.2293715087159928
0.3393565201557055
0.08160808951872765
1.0231890698461015
-0.030936235183923053
-0.0
0.03029848710093158
0.055450418967022436
0.010610327477210144
0.007510182926207109
-0.33288853375281785
0

-0.27434394211139596
-0.26288749443441134
-0.07811230151726521
-8.043563285906174
3.4410864112101076
10.488888888888896
-17.645478395867272
-11.04477387559809
1.472222602939082
-0.004108426893803578
-0.02437337262396136
0.008966574761644652
0.873769243136589
-86.64798923772005
-1.6974313689435565
-0.7752616057992759
0.54048336105764
0.0009354739812348547
-0.003770368278906895
0.04829146536786196
0.09827727707130741
-0.008425904082087769
-0.5634243056990293
0.025761567125210733
-0.9348441626948742
0.011876219905819227
-0.005790379593063014
9.924777276756473
0.3566821782430469
0.8928344139624462
0.8762074281629747
0.0029668770882767923
0.02777600796833552
0.02777600796833552
-0.6240552902270783
0.03525422017961116
-0.0015918365120417938
0.0012407241972418452
0.11650741456057462
0.6136008821034761
-0.01427621012562154
-0.011824738645236983
11.44176429227137
0.003404092748364821
-0.18877409775441487
0.1873004089174084
0.013027437164978638
0.020994356608616663
-0.09319482914979746
0.0122044

...

-1.6313297900771142
-21.829573840206805
-3.6307971828123464
-13.87332518627802
-1.246110376400765
-6.205289178040814
-2.7530184303486944
-6.036550751579829
-20.296690231289464
-31.885012664240634
-8.51256063922154
-3.421235118623196
2.3283283109303263
-14.20736067564249
-3.2567665746464325
-3.817459573534154
-2.732685940034025
-6.365134459493101
-3.8613891539048684
-2.061037309945191
-2.864441770412798
-7.624380618568554
-12.628985348410323
-1.559645846313252
-2.1640061511932753
-8.022250574762518
-3.376299631315608
-9.917912619868401
-6.495640469851856
-6.0878231086742
-14.760403474248708
-4.885079140032243
-2.467583666537521
-4.071136850997755
-26.54346447745467
-7.9256917958550295
-5.291484113438616
-4.384866661860798
-29.36192197888016
-5.473169498812311
-1.5279096637902088
-7.766440338636911
-6.69622853746416
-2.80745973103873
8.584402390281156
-4.225587462128823
-2.2425779175466936
-2.3273336296159006
-2.2254308297994103
15.409846553488945
-3.491377085628469
3.73534203987339
-0.9

-1.6052941176470588
-1.5937037037037038
-1.7129629629629628
-1.6052941176470588
-1.5543453510436431
-1.5543453510436431
-1.23990231990232
-2.397291280148423
-1.4559358288770055
-1.4264150943396228
-1.0175
-2.2377075098814228
-1.724935064935065
-2.4629159519725556
-1.9397463002114166
-1.7326004228329812
-2.58642735042735
-1.3456336405529954
-1.6757142857142857
-1.6937799043062203
-1.5220779220779221
-1.677537372147915
-1.2222727272727272
-2.811590909090909
-1.4697414395527604
-1.4291700404858299
-2.27511961722488
-4.123113006396588
-1.3116666666666668
-1.6609719839908519
-1.7454545454545454
-1.4429900332225913
-1.5253061224489795
-2.13375
-2.2807792207792206
-2.0241935483870965
-2.305691699604743
-2.0086206896551726
-1.6272727272727274
-1.3152727272727271
-1.6978181818181817
-1.5619810126582279
-2.0725942350332596
-1.3523636363636364
-0.965
-1.687878787878788
-1.8435804195804195
-1.932381864623244
-1.8069369369369368
-1.5663692307692307
-1.5907692307692307
-1.4208196721311475
-1.3878374

0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.18259999999999998
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0


-0.025748145242997705
0.12286115387564657
-0.987768813597923
0.24875633224111948
0.3133815242567246
-0.5553715778994525
-0.08742857142857141
0.2663402215642652
-0.8095051222069858
-0.32703245566805067
-0.4222872450076097
-0.34816093370124557
-0.07232157254510216
0.4345456636760984
-0.47944080939353456
-0.5200747798404902
-0.4163388430020601
-0.16351937448265363
-0.009062895622895625
-0.09718112808600624
-0.26704235369658935
-0.3222616042780749
-0.36056457992563135
-0.7153558551778922
-0.1104824561403508
-0.26647483178759773
-0.21078963641031637
-0.5795909060816705
-0.6568629214802849
-0.4983453084663609
-0.1859538526731559
0.1802822746642518
-0.08939325038191373
-0.8348825461088617
-0.22821840592417922
-0.07100858813700905
1.5034295461701126
-0.5618153421859038
0.5593320807371656
0.3889605108935815
-0.3214633685773043
-0.07281229129397726
0.08535136742544153
-0.48317256062362157
-0.42709046200406675
-0.14292413793103434
-1.4351392128007694
0.6332349332691686
0.14316626038487998
-0.3839

-0.11403720062227796
0.19150335885448344
-0.4290595065261615
0.1994042294270557
0.20214756200285766
-0.02999227023446077
0.45971961420222296
0.5314907711895949
-1.0120741651998306
-0.47716233142920605
0.7256106556857778
-0.23988293449041614
0.6792676767676767
-0.3950784429605481
-0.32339484399483
-0.5873832177004591
0.07784965713552194
-1.1269491258041677
-0.8445973315178432
0.4596807205228925
-0.5106793354793355
0.10236763641331689
0.05168641041498173
0.40966588298201234
0.14344927883176747
-0.30040612966463626
0.03461648892534869
-0.09943973426682273
0.2439788416711495
0.10111598005559747
0.21802659080500647
0.23154034886681962
-0.31204466028993877
-0.5865472570792334
-0.29850692897112163
-0.21928855933981994
0.31291509852830607
-0.012712477718359982
-0.012712477718359982
-0.4842977242943904
-0.46307607503607506
0.7456124619873568
-0.530399069997239
-0.34589718652665047
-0.1279537912502494
0.12237718474526532
-0.7579670446318861
-1.1641764385940856
0.6559298605711917
-0.0056292825832